<img width="800px" src="../fidle/img/header.svg"></img>

# <!-- TITLE --> [AE4] - Denoiser and classifier model
<!-- DESC --> Episode 4 : Construction of a denoiser and classifier model

<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - Building a multiple output model, able to **denoise** and **classify**
 - Understanding a more **advanced programming model**

The calculation needs being important, it is preferable to use a very simple dataset such as MNIST.  
The use of a GPU is often indispensable.

## What we're going to do :

 - Defining a multiple output model using Keras procedural programing model
 - Build the model
 - Train it
 - Follow the learning process
 
## Data Terminology :
- `clean_train`, `clean_test` for noiseless images 
- `noisy_train`, `noisy_test` for noisy images
- `class_train`, `class_test` for the classes to which the images belong 
- `denoised_test` for denoised images at the output of the model
- `classcat_test` for class prediction in model output (is a softmax)
- `classid_test` class prediction (ie: argmax of classcat_test)


## Step 1 - Init python stuff
### 1.1 - Init

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'torch'

import keras

import numpy as np
from skimage import io
import random

from modules.AE4_builder    import AE4_builder
from modules.MNIST          import MNIST
from modules.ImagesCallback import ImagesCallback

import fidle

# Init Fidle environment
run_id, run_dir, datasets_dir = fidle.init('AE4')

### 1.2 - Parameters
`prepared_dataset` : Filename of the prepared dataset (Need 400 Mo, but can be in ./data)  
`dataset_seed` : Random seed for shuffling dataset. 'None' mean using /dev/urandom  
`scale` : % of the dataset to use (1. for 100%)  
`latent_dim` : Dimension of the latent space  
`train_prop` : Percentage for train (the rest being for the test)
`batch_size` : Batch size  
`epochs` : Nb of epochs for training\
`fit_verbosity` is the verbosity during training : 0 = silent, 1 = progress bar, 2 = one line per epoch

scale=0.1, epochs=20  => 2' on a laptop


In [ ]:
prepared_dataset = './data/mnist-noisy.h5'
dataset_seed     = None

scale            = .1

train_prop       = .8
batch_size       = 128
epochs           = 10
fit_verbosity    = 1

Override parameters (batch mode) - Just forget this cell

In [ ]:
fidle.override('prepared_dataset', 'dataset_seed', 'scale')
fidle.override('train_prop', 'batch_size', 'epochs', 'fit_verbosity')

## Step 2 - Retrieve dataset
With our MNIST class, in one call, we can reload, rescale, shuffle and split our previously saved dataset :-)

In [ ]:
clean_train,clean_test, noisy_train,noisy_test, class_train,class_test = MNIST.reload_prepared_dataset(
                                                                                    scale      = scale, 
                                                                                    train_prop = train_prop,
                                                                                    seed       = dataset_seed,
                                                                                    shuffle    = True,
                                                                                    filename   = prepared_dataset )

## Step 3 - Build models

In [ ]:
builder = AE4_builder( ae={ 'latent_dim':10 }, cnn = { 'lc1':8, 'lc2':16, 'ld':100 } )

model = builder.create_model()


In [ ]:
model.compile(optimizer='rmsprop', 
              loss={'ae':'binary_crossentropy', 'classifier':'sparse_categorical_crossentropy'},
              loss_weights={'ae':1., 'classifier':1.},
              metrics={'classifier':'accuracy'} )

In [ ]:
# keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

## Step 4 - Train
20' on a CPU  
1'12 on a GPU (V100, IDRIS)

In [ ]:
# ---- Callback : Images
#
fidle.utils.mkdir( run_dir + '/images')
filename = run_dir + '/images/image-{epoch:03d}-{i:02d}.jpg'

encoder = model.get_layer('ae').get_layer('encoder')
decoder = model.get_layer('ae').get_layer('decoder')

callback_images = ImagesCallback(filename, x=clean_test[:5], encoder=encoder,decoder=decoder)

In [ ]:
chrono = fidle.Chrono()
chrono.start()

history = model.fit(noisy_train, [clean_train, class_train],
                 batch_size      = batch_size,
                 epochs          = epochs,
                 verbose         = fit_verbosity,
                 validation_data = (noisy_test, [clean_test, class_test]),
                 callbacks       = [ callback_images ]  )

chrono.show()

Save model weights

In [ ]:
os.makedirs(f'{run_dir}/models', exist_ok=True)

model.save_weights(f'{run_dir}/models/model.weights.h5')

## Step 5 - History

In [ ]:
fidle.scrawler.history(history,  plot={'Loss':['loss', 'val_loss'],
                                 'Accuracy':['classifier_accuracy','val_classifier_accuracy']}, save_as='01-history')

## Step 6 - Denoising progress

In [ ]:
imgs=[]
for epoch in range(0,epochs,4):
    for i in range(5):
        filename = run_dir + '/images/image-{epoch:03d}-{i:02d}.jpg'.format(epoch=epoch, i=i)
        img      = io.imread(filename)
        imgs.append(img)      

fidle.utils.subtitle('Real images (clean_test) :')
fidle.scrawler.images(clean_test[:5], None, indices='all', columns=5, x_size=2,y_size=2, interpolation=None, save_as='02-original-real')

fidle.utils.subtitle('Noisy images (noisy_test) :')
fidle.scrawler.images(noisy_test[:5], None, indices='all', columns=5, x_size=2,y_size=2, interpolation=None, save_as='03-original-noisy')

fidle.utils.subtitle('Evolution during the training period (denoised_test) :')
fidle.scrawler.images(imgs, None, indices='all', columns=5, x_size=2,y_size=2, interpolation=None, y_padding=0.1, save_as='04-learning')

fidle.utils.subtitle('Noisy images (noisy_test) :')
fidle.scrawler.images(noisy_test[:5], None, indices='all', columns=5, x_size=2,y_size=2, interpolation=None, save_as=None)

fidle.utils.subtitle('Real images (clean_test) :')
fidle.scrawler.images(clean_test[:5], None, indices='all', columns=5, x_size=2,y_size=2, interpolation=None, save_as=None)


## Step 7 - Evaluation
**Note :** We will use the following data:\
`clean_train`, `clean_test` for noiseless images \
`noisy_train`, `noisy_test` for noisy images\
`class_train`, `class_test` for the classes to which the images belong \
`denoised_test` for denoised images at the output of the model\
`classcat_test` for class prediction in model output (is a softmax)\
`classid_test` class prediction (ie: argmax of classcat_test)
 
### 7.1 - Reload our model (weights)

In [ ]:
builder = AE4_builder( ae={ 'latent_dim':10 }, cnn = { 'lc1':8, 'lc2':16, 'ld':100 } )

model = builder.create_model()

model.load_weights(f'{run_dir}/models/model.weights.h5')

### 7.2 - Let's make a prediction
Note that our model will returns 2 outputs : **denoised images** from output 1 and **class prediction** from output 2

In [ ]:
outputs = model.predict(noisy_test, verbose=0)

denoised = outputs['ae']
classcat = outputs['classifier']

print('Denoised images   (denoised_test) shape : ', denoised.shape)
print('Predicted classes (classcat_test) shape : ', classcat.shape)

### 7.3 - Denoised images 

In [ ]:
i=random.randint(0,len(denoised)-8)
j=i+8

fidle.utils.subtitle('Noisy test images (input):')
fidle.scrawler.images(noisy_test[i:j], None, indices='all', columns=8, x_size=2,y_size=2, interpolation=None, save_as='05-test-noisy')

fidle.utils.subtitle('Denoised images (output):')
fidle.scrawler.images(denoised[i:j], None, indices='all', columns=8, x_size=2,y_size=2, interpolation=None, save_as='06-test-predict')

fidle.utils.subtitle('Real test images :')
fidle.scrawler.images(clean_test[i:j], None, indices='all', columns=8, x_size=2,y_size=2, interpolation=None, save_as='07-test-real')

### 7.4 - Class prediction
Note: The evaluation requires the noisy images as input (noisy_test) and the 2 expected outputs:
 - the images without noise (clean_test)
 - the classes (class_test)

In [ ]:
# We need to (re)compile our resurrected model (to specify loss and metrics)
#
model.compile(optimizer='rmsprop', 
              loss={'ae':'binary_crossentropy', 'classifier':'sparse_categorical_crossentropy'},
              loss_weights={'ae':1., 'classifier':1.},
              metrics={'classifier':'accuracy'} )


# Get an evaluation
#
score = model.evaluate(noisy_test, [clean_test, class_test], verbose=0)

# And show results
#
fidle.utils.subtitle("Accuracy :")
print(f'Classification accuracy : {score[1]:4.4f}')

fidle.utils.subtitle("Few examples :")
classid_test  = np.argmax(classcat, axis=-1)
fidle.scrawler.images(noisy_test, class_test, range(0,200), columns=12, x_size=1, y_size=1, y_pred=classid_test, save_as='04-predictions')

In [ ]:
fidle.end()

---
<img width="80px" src="../fidle/img/logo-paysage.svg"></img>